## To test the Python API


In [1]:
from lilac.db_manager import get_dataset, set_default_dataset_cls
from lilac.data.dataset_duckdb import DatasetDuckDB
from lilac.signals.concept_scorer import ConceptScoreSignal
from lilac.signals.default_signals import register_default_signals

register_default_signals()
set_default_dataset_cls(DatasetDuckDB)

dataset = get_dataset('local', 'legal-clauses')

dataset.compute_signal(
  ConceptScoreSignal(namespace='lilac', concept_name='legal-termination', embedding='sbert'),
  'clause_text')

Computing signal "signal_name='concept_score' embedding='sbert' namespace='lilac' concept_name='legal-termination' draft='main' num_negative_examples=100" took 0.339s.
Wrote signal manifest to ./data/datasets/local/legal-clauses/clause_text/sbert/embedding/lilac/legal-termination/v33/signal_manifest.json
